# DM displays

In [ ]:
# Imports
%matplotlib inline
import os
from astropy.io import fits
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, LogNorm, TwoSlopeNorm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import Asterix_root
from Asterix.utils import get_data_dir, read_parameter_file
from Asterix.optics import Pupil, Coronagraph, DeformableMirror, Testbed
from Asterix.main_THD import THD2

In [ ]:
# Some setup for pretty plotting
mpl.rc('image', origin='lower',   # Put the origin in the lower left corner.
       interpolation=None)        # Do not interpolate between pixels in the display.

cenm = TwoSlopeNorm(vcenter=0)    # Create normalization where 0 is always the center of your cmap

In [ ]:
# Some functions for colormap setting
# Source: https://towardsdatascience.com/beautiful-custom-colormaps-with-matplotlib-5bab3d1f0e72
def hex_to_rgb(value):
    '''
    Converts hex to rgb colours
    value: string of 6 characters representing a hex colour.
    Returns: list length 3 of RGB values'''
    value = value.strip("#") # removes hash symbol if present
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_dec(value):
    '''
    Converts rgb to decimal colours (i.e. divides each value by 256)
    value: list (length 3) of RGB values
    Returns: list (length 3) of decimal values'''
    return [v/256 for v in value]


def get_continuous_cmap(hex_list, float_list=None):
    ''' creates and returns a color map that can be used in heat map figures.
        If float_list is not provided, colour map graduates linearly between each color in hex_list.
        If float_list is provided, each color in hex_list is mapped to the respective location in float_list. 
        
        Parameters
        ----------
        hex_list: list of hex code strings
        float_list: list of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        
        Returns
        ----------
        colour map'''
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
        
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=256)
    return cmp


hex_list = ['#020202', '#00ca00', '#cf00b6', '#bde892', '#1700ee', '#f7a3e1', '#373f50', '#fdfdfb']

In [ ]:
# Create testbed instance
parameter_file_ex = os.path.join(Asterix_root, "Example_param_file.ini")
config = read_parameter_file(parameter_file_ex)
model_dir = os.path.join(get_data_dir(), "Model_local")

thd2 = THD2(config, model_dir)

In [ ]:
print("####################################")
print(f"# Coronagraph used: {thd2.corono.corona_type}")
print(f"# Wavelength: {thd2.wavelength_0} m = {thd2.wavelength_0 * 1e9} nm")
print("####################################")

In [ ]:
print(type(thd2.DM3.DM_pushact))
print(thd2.DM3.DM_pushact.shape)
dm3 = np.sum(thd2.DM3.DM_pushact, axis=0)
print(dm3.shape)

In [ ]:
plt.figure(figsize=(9, 9))
plt.imshow(dm3, cmap='inferno')
plt.title('All actuators on DM3', size=20)
plt.colorbar()

In [ ]:
print(type(thd2.entrancepupil))
print(type(thd2.entrancepupil.pup))
pup = thd2.entrancepupil.pup
print(pup.shape)

In [ ]:
plt.imshow(pup, cmap='Greys_r')
plt.title('Pupil mask', size=20)
plt.colorbar()

In [ ]:
# Apologies for the colormap, I tried making it as hidious as possible to be able to see all details
plt.figure(figsize=(9, 9))
plt.title('Pupil mask overlaid on DM3')
plt.imshow(dm3, cmap=get_continuous_cmap(hex_list, float_list=[0, 0.05, 0.1, 0.92, 0.94, 0.96, 0.98, 1]))
plt.colorbar()
plt.imshow(pup, cmap='Greys_r', alpha=0.5)

In [ ]:
# Zoom in
cen = int(pup.shape[0] / 2)
bbox = 60

plt.figure(figsize=(9, 9))
plt.title('Pupil mask overlaid on DM3')
plt.imshow(dm3[cen-bbox:cen+bbox, cen-bbox:cen+bbox], cmap=get_continuous_cmap(hex_list, float_list=[0, 0.05, 0.1, 0.92, 0.94, 0.96, 0.98, 1]))
plt.colorbar()
plt.imshow(pup[cen-bbox:cen+bbox, cen-bbox:cen+bbox], cmap='Greys_r', alpha=0.5)

In [ ]:
# Pick only one actuator
act = 48

cen = int(pup.shape[0] / 2)
bbox = 60

plt.figure(figsize=(9, 9))
plt.title('Pupil mask overlaid on DM3')
plt.imshow(thd2.DM3.DM_pushact[act][cen-bbox:cen+bbox, cen-bbox:cen+bbox], cmap=get_continuous_cmap(hex_list, float_list=[0, 0.05, 0.1, 0.92, 0.94, 0.96, 0.98, 1]))
plt.colorbar()
plt.imshow(pup[cen-bbox:cen+bbox, cen-bbox:cen+bbox], cmap='Greys_r', alpha=0.5)